# ISDF with PYSCF and Octopus

Octopus gives erroneous results for the serial ISDF implementation. As such, this notebook:
* Reimplements ISDF using PYSCF as a base
    * See prior notebooks [isdf_vectors.ipynb] and [qr_on_orbitals.ipynb]
* Does QR decomposition instead of kmeans
* Parses in Octopus wave functions, and see if one can get good results



In [3]:
"""Build Benzene molecule with a minimal GTO basis, using PYSCF
"""
%load_ext autoreload
%autoreload 2

import numpy as np
from pathlib import Path

from isdf_prototypes.clean_isdf import benzene_from_pyscf, bohr_to_ang, \
    construct_interpolation_vectors_more_parts, \
    approximate_product_basis, \
    error_l2, mean_norm, \
    find_interpolation_points_factory
from isdf_prototypes.math_ops import face_splitting_product


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
""" Get wave functions, grid and density.
* PYSCF 
* Read from file, for Octopus
"""
method = 'pyscf'

if method == 'pyscf':
    output_root = Path('../pyscf_nov2024_outputs')
    data: dict = benzene_from_pyscf(output_root, [10, 10, 10])
    print(data.keys())
    # assert [data] == ['wfs', 'rho', 'cube_grid'], 'Expected output data from pyscf calculation'
    
    # Add real-space points, and volume element
    data.update({'grid_points': data['cube_grid'].get_coords()})
    data.update({'dV': data['cube_grid'].get_volume_element()})

elif method == 'octopus':
    output_root = Path('../oct_nov2024_outputs')
    # Require wfs, rho, grid_points and dV - volume element
    print('Add read in and reshaping')
    
else:
    raise ValueError(f'Erroneous GS Method: {method}')


converged SCF energy = -229.930646528207
dict_keys(['wfs', 'rho', 'cube_grid'])


In [62]:
""" Compute interpolation indices

QR Approach 1
1. Sample from a Gaussian distribution to form a Gaussian matrix
  - Apply QR decomposition to G, and take Q, such that the matrix is orthogonalised
2. Contract phi and G over the state index. Take the face-splitting product of this to form Z_tilde

QR Approach 3
1. Compute interpolation indices using the QR Approach, BUT without orthogonalising the Gaussian
sampling matrices

KMeans Approach
"""
from isdf_prototypes.visualise import write_xyz

n_int = 200
interpolation_method = 'kmeans'

n_grid_points = data['grid_points'].shape[0]
n_states = data['wfs'].shape[1]
assert data['wfs'].shape[0] == n_grid_points, 'Shape of wfs inconsistent with the grid'

indices = find_interpolation_points_factory(interpolation_method)(n_int, **data)

# Output grid points from indices to .xyz
with open(file= output_root / f"indices_{interpolation_method}.xyz", mode='w') as fid:
    string = write_xyz(['B']*n_int, data['grid_points'][indices]* bohr_to_ang)
    fid.write(string)
    

Interpolation Method is kmeans


In [63]:
""" Compute ISDF Vectors
"""
isdf_vectors = construct_interpolation_vectors_more_parts(data['wfs'], indices)
assert isdf_vectors.shape == (n_grid_points, n_int)
    

In [64]:
""" Compute full product matrix and approximate product matrix
"""
# Construct product basis matrix
z = face_splitting_product(data['wfs'])
assert z.shape == (n_grid_points, n_states**2)

# Approximate product basis amtric
z_isdf = approximate_product_basis(data['wfs'], indices, isdf_vectors)
assert z_isdf.shape == (n_grid_points, n_states**2)

error = error_l2(z, z_isdf, data['dV'])
rel_l2_error = error['mean'] / mean_norm(z, data['dV'])

print(f"{indices.size}, {error['min']:.2e}, {error['max']:.2e}, {error['mean']:.2e}, {rel_l2_error:.2e}")

if method == 'pyscf':
    print('Outputting a subset of the product states for visualisation')
    # Note, this is hard-coded in `benzene_from_pyscf`
    nx, ny, nz  = 10, 10, 10
    
    print()
    
    # Total range from index 0, to n_states**2-1
    
    for ij in [0, 25, 80, n_states**2-1]:
        # Exact product state
        fname = output_root / f'z_{ij}.cube'
        data['cube_grid'].write(field=z[:, ij].reshape(nx, ny, nz) , fname=fname.as_posix())
        # Approximate product state
        fname = output_root / f'zisdf_{ij}.cube'
        data['cube_grid'].write(field=z_isdf[:, ij].reshape(nx, ny, nz) , fname=fname.as_posix())



200, 7.77e-10, 8.54e-08, 2.41e-08, 1.65e-05
Outputting a subset of the product states for visualisation


## Results

### KMeans

Repeat each run 3 times, and highlight the most favourable.
Note, for 200 centroids $[CC^T]^{-1}$ fails `scipy.linalg.issymmetric(inv_cct, rtol=1.e-4)`. Have a think about why that is, and look at the condition number.
TODO - Plot this convergence


| N Interpolation Points | Min Error                   | Max Error                 | Mean Error               | Relative (L2) Error       | Notes                                                           |
|------------------------|-----------------------------|---------------------------|--------------------------|---------------------------|-----------------------------------------------------------------|
| 10                     | 1.38e-4  1.07e-4   1.27e-3  | 2.58e-3  2.57e-3  2.72e-3 | 1.09e-3  1.04e-3 1.07e-3 | 0.746    0.713    0.732   | Clearly the same functions, but not full visual agreement       |
| 25                     | 7.58e-5  7.50e-5   6.75e-5  | 2.02e-3  2.12e-3  2.18e-3 | 6.82e-4  6.95e-4 7.07e-4 | 0.466    0.475    0.48    | Visual agreement improved, but clearly differ                   | 
| 50                     | 3.10e-5  4.61e-5   4.75e-5  | 1.14e-3  1.86e-3  1.29e-3 | 2.61e-4  3.31e-4 3.03e-4 | 0.178    0.23     0.21    | Visual agreement to the point where the functions look the same |
| 100                    | 2.61e-6  1.31e-6   1.89e-6  | 3.15e-5  3.97e-5  2.38e-5 | 1.11e-5  7.81e-6 8.49e-6 | 7.60e-3  5.33e-3  5.80e-3 | Didn't check - assume fine                                      |
| 200                    | 1.14e-9  1.85e-9   7.77e-10 | 8.76e-8  1.05e-7  8.54e-8 | 2.50e-8  2.81e-8 2.41e-8 | 1.71e-5  1.92e-5  1.65e-5 | Perfect visual agreement                                        |


### Sub-sampling with Orthogonal G Matrix

Repeat each run 3 times, and highlight the most favourable

| N Interpolation Points | Min Error | Max Error | Mean Error | Relative (L2) Error |
|------------------------|-----------|-----------|------------|---------------------|
| 10                     |           |           |            |                     | 
| 25                     |           |           |            |                     | 
| 50                     |           |           |            |                     | 
| 100                    |           |           |            |                     | 
| 200                    |           |           |            |                     | 

The same as the above, but I do:
$$
\tilde{Z}_{\alpha \beta}(\mathbf{r}) =
       \left(\sum_{i=1}^m \varphi_i(\mathbf{r}) G_{i \alpha}^{\varphi}\right)
       \left(\sum_{j=1}^n \varphi_j(\mathbf{r}) G_{j \beta}^\psi\right)
$$
rather than:
$$
\tilde{Z}_{\alpha \beta}(\mathbf{r}) =
       \left(\sum_{i=1}^m \varphi_i(\mathbf{r}) G_{i \alpha}^{\varphi}\right)
       \left(\sum_{i=1}^m \varphi_i(\mathbf{r}) G_{i \alpha}^{\varphi}\right)
$$

i.e use two different sampling matrices on the same set of KS states.


### Sub-sampling with Non-Orthogonal G Matrix

Repeat each run 3 times, and highlight the most favourable

| N Interpolation Points | Min Error | Max Error | Mean Error | Relative (L2) Error |
|------------------------|-----------|-----------|------------|---------------------|
| 10                     |           |           |            |                     | 
| 25                     |           |           |            |                     | 
| 50                     |           |           |            |                     | 
| 100                    |           |           |            |                     | 
| 200                    |           |           |            |                     | 

